In [1]:
%cd /home/q123/Desktop/explo

import torch 
import gpytorch 
import logging
import logging.config

from src.helpers import setup_experiment
from src.trainer import Trainer
from src.gibo.optim import GIBOptimizer
from src.optim import BOptimizer
from src.config import get_configs

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_name = "Swimmer-v2"
#kernel_name = "rbf"
kernel_name = "linearstate"

env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
model,objective_env = setup_experiment(env_config,kernel_config,likelihood_config)

trainer_config = {
        "n_steps" :100,
        "report_freq":10,
        "save_best":False,
}

### Chose optimizer 
#optimizer = BOptimizer(**optimizer_config)
optimizer = GIBOptimizer(model,**optimizer_config)
trainer = Trainer(model,objective_env,optimizer,**trainer_config)
rslt= trainer.run()

/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment Swimmer-v2 is out of date. You should consider upgrading to version `v3` with the environment ID `Swimmer-v3`.
  logger.warn(


MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
 Gibo will use 32 last points to fit GP and 16 info samples
current 0.04861173778772354 / max 0.14201919734477997 /batch_mean 0.07082841545343399 /batch_max 0.10378764569759369 
##############################
covar_lengthscale max 0.00046355690574273467 / min -8.736568815947976e-06
##############################
MLL : 1.3640493154525757
current 0.06037099286913872 / max 0.14201919734477997 /batch_mean 0.0756186917424202 /batch_max 0.10089805722236633 
##############################
covar_lengthscale max 0.00043562939390540123 / min -9.74920294538606e-06
##############################
MLL : 1.3761543035507202
current 0.06293421238660812 / max 0.14201919734477997 /batch_mean 0.06903553009033203 /batch_max 0.07697758078575134 
##############################
covar_lengthscale max 0.00042730089626275003 / min -6.174176814965904e-05
##############################
MLL : 1.377789855003357


KeyboardInterrupt: 